Library Imports

In [214]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

Data Import

In [215]:
df_full = pd.read_csv(r'C:\Users\jacks\Documents\GitHub\Football-Analytics\Data\collatedData17-20.csv')
df = df_full


Pre-Processing

Filter dataframe to player data from just the years 17/18 and 20/21.

Filter this further to Just those in the forward positions

In [216]:
roles=['FW','FWDF','FWMF','MFFW']
df = df[((df['y2017-18_Pos'].notnull()) & (df['y2020-21_Pos'].isin(roles)))]
df

,Player,y2020-21_Rk,y2020-21_Nation,y2020-21_Pos,y2020-21_Squad,y2020-21_Age,y2020-21_Born,y2020-21_MP,y2020-21_Starts,y2020-21_Min,...,y2017-18_Crs.1,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%
6,Adam Lallana\99813635,258,eng ENG,MFFW,Brighton,32,1988,30,16,1596,...,6.0,5.0,3.0,0.0,0.0,0.0,32.0,3.0,6.0,33.3
9,Ademola Lookman\7c104bb7,281,eng ENG,FWMF,Fulham,22,1997,34,31,2747,...,0.0,2.0,2.0,0.0,0.0,0.0,16.0,1.0,1.0,50.0
15,Aleksandar MitroviÃÂ\3925dbd6,329,rs SRB,FW,Fulham,25,1994,27,13,1402,...,1.0,1.0,0.0,0.0,0.0,0.0,9.0,4.0,9.0,30.8
16,Alex Iwobi\6ca5ec4b,216,ng NGA,FWDF,Everton,24,1996,30,17,1542,...,29.0,12.0,21.0,0.0,0.0,0.0,149.0,7.0,21.0,25.0
18,Alex Oxlade-Chamberlain\38c7feef,364,eng ENG,FWMF,Liverpool,26,1993,13,2,253,...,40.0,20.0,23.0,0.0,0.0,0.0,153.0,16.0,31.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,Theo Walcott\1ecb65be,499,eng ENG,FW,Everton,31,1989,1,0,13,...,40.0,14.0,12.0,0.0,0.0,0.0,100.0,8.0,29.0,21.6
525,Wilfried Zaha\b2bc3b1f,526,ci CIV,FW,Crystal Palace,27,1992,30,29,2612,...,85.0,19.0,24.0,4.0,0.0,0.0,186.0,2.0,49.0,3.9
530,Willian\8b9ebd03,516,br BRA,FWMF,Arsenal,31,1988,25,16,1406,...,51.0,12.0,24.0,1.0,0.0,0.0,142.0,11.0,23.0,32.4
533,Xherdan Shaqiri\6421ec64,435,ch SUI,MFFW,Liverpool,28,1991,14,5,556,...,101.0,30.0,16.0,0.0,0.0,0.0,209.0,10.0,32.0,23.8


Label the Rows.
At the moment it is very simple: 8 goals or over is a 1.

This is a first attempt, so is simple by design

In [217]:
pd.options.mode.chained_assignment = None  # default='warn'
def calc_label(row):
    if row['y2020-21_Gls'] < 6:
        value = 0
    else:
        value = 1
    return value

df['label'] = df.apply(calc_label, axis=1)
df['label'].value_counts()

0    52
1    26
Name: label, dtype: int64

Remove data columns for the 20/21 season, so that the models can train on the 17/18 season data

In [218]:
df = df[df.columns[-150:]]
df

,y2017-18_Nation,y2017-18_Pos,y2017-18_Squad,y2017-18_Age,y2017-18_Born,y2017-18_MP,y2017-18_Starts,y2017-18_Min,y2017-18_90s,y2017-18_Gls,...,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%,label
6,eng ENG,MFFW,Liverpool,29.0,1988.0,12.0,1.0,308.0,3.4,0.0,...,5.0,3.0,0.0,0.0,0.0,32.0,3.0,6.0,33.3,0
9,eng ENG,FWMF,Everton,19.0,1997.0,7.0,1.0,159.0,1.8,0.0,...,2.0,2.0,0.0,0.0,0.0,16.0,1.0,1.0,50.0,0
15,rs SRB,FW,Newcastle Utd,22.0,1994.0,6.0,0.0,95.0,1.1,1.0,...,1.0,0.0,0.0,0.0,0.0,9.0,4.0,9.0,30.8,0
16,ng NGA,FWMF,Arsenal,21.0,1996.0,26.0,22.0,1830.0,20.3,3.0,...,12.0,21.0,0.0,0.0,0.0,149.0,7.0,21.0,25.0,0
18,eng ENG,MFFW,Liverpool,23.0,1993.0,32.0,14.0,1493.0,16.6,3.0,...,20.0,23.0,0.0,0.0,0.0,153.0,16.0,31.0,34.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,eng ENG,FW,Everton,28.0,1989.0,14.0,13.0,1152.0,12.8,3.0,...,14.0,12.0,0.0,0.0,0.0,100.0,8.0,29.0,21.6,0
525,ci CIV,FWMF,Crystal Palace,24.0,1992.0,29.0,28.0,2549.0,28.3,9.0,...,19.0,24.0,4.0,0.0,0.0,186.0,2.0,49.0,3.9,1
530,br BRA,FWMF,Chelsea,28.0,1988.0,36.0,20.0,1890.0,21.0,6.0,...,12.0,24.0,1.0,0.0,0.0,142.0,11.0,23.0,32.4,0
533,ch SUI,FWMF,Stoke City,25.0,1991.0,36.0,36.0,3039.0,33.8,8.0,...,30.0,16.0,0.0,0.0,0.0,209.0,10.0,32.0,23.8,0


Remove unneccessary fields from the 17/28 data

In [219]:
df = df.drop(['y2017-18_Nation',
                                'y2017-18_Pos',
                                'y2017-18_Squad']
                                ,1)
df

,y2017-18_Age,y2017-18_Born,y2017-18_MP,y2017-18_Starts,y2017-18_Min,y2017-18_90s,y2017-18_Gls,y2017-18_Ast,y2017-18_G-PK,y2017-18_PK,...,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%,label
6,29.0,1988.0,12.0,1.0,308.0,3.4,0.0,0.0,0.0,0.0,...,5.0,3.0,0.0,0.0,0.0,32.0,3.0,6.0,33.3,0
9,19.0,1997.0,7.0,1.0,159.0,1.8,0.0,0.0,0.0,0.0,...,2.0,2.0,0.0,0.0,0.0,16.0,1.0,1.0,50.0,0
15,22.0,1994.0,6.0,0.0,95.0,1.1,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,9.0,4.0,9.0,30.8,0
16,21.0,1996.0,26.0,22.0,1830.0,20.3,3.0,5.0,3.0,0.0,...,12.0,21.0,0.0,0.0,0.0,149.0,7.0,21.0,25.0,0
18,23.0,1993.0,32.0,14.0,1493.0,16.6,3.0,7.0,3.0,0.0,...,20.0,23.0,0.0,0.0,0.0,153.0,16.0,31.0,34.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,28.0,1989.0,14.0,13.0,1152.0,12.8,3.0,3.0,3.0,0.0,...,14.0,12.0,0.0,0.0,0.0,100.0,8.0,29.0,21.6,0
525,24.0,1992.0,29.0,28.0,2549.0,28.3,9.0,3.0,9.0,0.0,...,19.0,24.0,4.0,0.0,0.0,186.0,2.0,49.0,3.9,1
530,28.0,1988.0,36.0,20.0,1890.0,21.0,6.0,7.0,5.0,1.0,...,12.0,24.0,1.0,0.0,0.0,142.0,11.0,23.0,32.4,0
533,25.0,1991.0,36.0,36.0,3039.0,33.8,8.0,7.0,8.0,0.0,...,30.0,16.0,0.0,0.0,0.0,209.0,10.0,32.0,23.8,0


Normalise Data using MinMax Scaling for each column.

For future development stages it may be worth leaving some columns out from this stage, aswell as adding extra columns based on the data available, but this is a first run, and so the aim is to keep things simple.

In [220]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns= df.columns)

#df_normalized.dropna()
for column in df_normalized:
    df_normalized[column] = df_normalized[column].fillna(df_normalized[column].mean())


df_normalized

,y2017-18_Age,y2017-18_Born,y2017-18_MP,y2017-18_Starts,y2017-18_Min,y2017-18_90s,y2017-18_Gls,y2017-18_Ast,y2017-18_G-PK,y2017-18_PK,...,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%,label
0,0.928571,0.142857,0.297297,0.027027,0.092341,0.091667,0.00000,0.000000,0.000000,0.0,...,0.096154,0.047619,0.000000,0.0,0.0,0.125000,0.011494,0.022989,0.333,0.0
1,0.214286,0.785714,0.162162,0.027027,0.046325,0.047222,0.00000,0.000000,0.000000,0.0,...,0.038462,0.031746,0.000000,0.0,0.0,0.062500,0.003831,0.003831,0.500,0.0
2,0.428571,0.571429,0.135135,0.000000,0.026560,0.027778,0.03125,0.000000,0.032258,0.0,...,0.019231,0.000000,0.000000,0.0,0.0,0.035156,0.015326,0.034483,0.308,0.0
3,0.357143,0.714286,0.675676,0.594595,0.562384,0.561111,0.09375,0.454545,0.096774,0.0,...,0.230769,0.333333,0.000000,0.0,0.0,0.582031,0.026820,0.080460,0.250,0.0
4,0.500000,0.500000,0.837838,0.378378,0.458308,0.458333,0.09375,0.636364,0.096774,0.0,...,0.384615,0.365079,0.000000,0.0,0.0,0.597656,0.061303,0.118774,0.340,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,0.857143,0.214286,0.351351,0.351351,0.352996,0.352778,0.09375,0.272727,0.096774,0.0,...,0.269231,0.190476,0.000000,0.0,0.0,0.390625,0.030651,0.111111,0.216,0.0
74,0.571429,0.428571,0.756757,0.756757,0.784435,0.783333,0.28125,0.272727,0.290323,0.0,...,0.365385,0.380952,0.666667,0.0,0.0,0.726562,0.007663,0.187739,0.039,1.0
75,0.857143,0.142857,0.945946,0.540541,0.580914,0.580556,0.18750,0.636364,0.161290,0.2,...,0.230769,0.380952,0.166667,0.0,0.0,0.554688,0.042146,0.088123,0.324,0.0
76,0.642857,0.357143,0.945946,0.972973,0.935763,0.936111,0.25000,0.636364,0.258065,0.0,...,0.576923,0.253968,0.000000,0.0,0.0,0.816406,0.038314,0.122605,0.238,0.0


Creating Training / Testing Datasets from the normalised data

In [221]:
from sklearn.model_selection import train_test_split
y = df[['label']]
X = df_normalized.drop(['label'],1)


print('Length of x: ',len(X))
print('Length of y: ',len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_test.fillna(X_train.mean(), inplace=True)
X_train.fillna(X_train.mean(), inplace=True)


print('Length of X_train: ',len(X_train))
print('Length of X_test: ',len(X_test))
print('Length of y_train: ',len(y_train))
print('Length of y_test: ',len(y_test))

Length of x:  78
Length of y:  78
Length of X_train:  54
Length of X_test:  24
Length of y_train:  54
Length of y_test:  24


Model Development

SVM Classifier

In [222]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

y_pred = svclassifier.predict(X_test)
#from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_pred,y_test))
print(classification_report(y_pred,y_test))

f1_score(y_test, y_pred, zero_division=1)


[[11  5]
 [ 1  7]]
              precision    recall  f1-score   support

           0       0.92      0.69      0.79        16
           1       0.58      0.88      0.70         8

    accuracy                           0.75        24
   macro avg       0.75      0.78      0.74        24
weighted avg       0.81      0.75      0.76        24



c:\Users\jacks\env\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7000000000000001

In [223]:
fn = r'C:\Users\jacks\Documents\GitHub\Football-Analytics\Models\SVM01.pkl'
pickle.dump(svclassifier, open(fn, 'wb'))

RandomForest Classification

In [224]:
RFclassifier = RandomForestClassifier()
RFclassifier.fit(X_train, y_train)

y_pred = RFclassifier.predict(X_test)


print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[10  2]
 [ 4  8]]
              precision    recall  f1-score   support

           0       0.71      0.83      0.77        12
           1       0.80      0.67      0.73        12

    accuracy                           0.75        24
   macro avg       0.76      0.75      0.75        24
weighted avg       0.76      0.75      0.75        24



c:\Users\jacks\env\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [225]:
fn = r'C:\Users\jacks\Documents\GitHub\Football-Analytics\Models\RMF01.pkl'
pickle.dump(RFclassifier, open(fn, 'wb'))

Naive Bayes

In [226]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[ 7  5]
 [ 2 10]]
              precision    recall  f1-score   support

           0       0.78      0.58      0.67        12
           1       0.67      0.83      0.74        12

    accuracy                           0.71        24
   macro avg       0.72      0.71      0.70        24
weighted avg       0.72      0.71      0.70        24



c:\Users\jacks\env\lib\site-packages\sklearn\naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [227]:
fn = r'C:\Users\jacks\Documents\GitHub\Football-Analytics\Models\GNB01.pkl'
pickle.dump(gnb, open(fn, 'wb'))

K-NearestNeighbour

In [228]:
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[11  1]
 [11  1]]
              precision    recall  f1-score   support

           0       0.50      0.92      0.65        12
           1       0.50      0.08      0.14        12

    accuracy                           0.50        24
   macro avg       0.50      0.50      0.39        24
weighted avg       0.50      0.50      0.39        24



c:\Users\jacks\env\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


In [229]:
neigh = KNeighborsClassifier(n_neighbors=7)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[12  0]
 [ 7  5]]
              precision    recall  f1-score   support

           0       0.63      1.00      0.77        12
           1       1.00      0.42      0.59        12

    accuracy                           0.71        24
   macro avg       0.82      0.71      0.68        24
weighted avg       0.82      0.71      0.68        24



c:\Users\jacks\env\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


In [230]:
fn = r'C:\Users\jacks\Documents\GitHub\Football-Analytics\Models\KNN01.pkl'
pickle.dump(neigh, open(fn, 'wb'))

Analysis

In [231]:


y_pred = svclassifier.predict(X_test)


out = y_test
out['pred'] = y_pred

def createOutputDF(df):
    outputmatrix=[]
    for i, row in df.iterrows():
        outputmatrix.append(df_full.loc[[i]].values.flatten().tolist())
    df_output = pd.DataFrame(outputmatrix,columns=df_full.columns)
    return df_output

df_TP = out[((out['label']==1) & (out['pred']==1))]
df_FP = out[((out['label']==0) & (out['pred']==1))]
df_TN = out[((out['label']==0) & (out['pred']==0))]
df_FN = out[((out['label']==1) & (out['pred']==0))]
df_TP = createOutputDF(df_TP)
df_FP = createOutputDF(df_FP)
df_TN = createOutputDF(df_TN)
df_FN = createOutputDF(df_FN)


True Positives

In [232]:
df_TP

,Player,y2020-21_Rk,y2020-21_Nation,y2020-21_Pos,y2020-21_Squad,y2020-21_Age,y2020-21_Born,y2020-21_MP,y2020-21_Starts,y2020-21_Min,...,y2017-18_Crs.1,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%
0,Chris Wood\4e9a0555,521,nz NZL,FW,Burnley,28,1991,33,32,2741,...,7.0,4.0,0.0,0.0,0.0,0.0,78.0,99.0,161.0,38.1
1,Richarlison\fa031b34,398,br BRA,FW,Everton,23,1997,34,33,2861,...,49.0,35.0,37.0,2.0,0.0,0.0,212.0,114.0,167.0,40.6
2,Harry Kane\21a66f6a,236,eng ENG,FW,Tottenham,27,1993,35,35,3082,...,24.0,7.0,11.0,1.0,0.0,0.0,123.0,71.0,112.0,38.8
3,Gabriel Jesus\b66315ae,224,br BRA,FW,Manchester City,23,1997,29,22,2063,...,8.0,13.0,17.0,0.0,0.0,0.0,79.0,21.0,61.0,25.6
4,Mohamed Salah\e342ad68,422,eg EGY,FW,Liverpool,28,1992,37,34,3078,...,50.0,13.0,13.0,1.0,0.0,0.0,218.0,19.0,58.0,24.7
5,Wilfried Zaha\b2bc3b1f,526,ci CIV,FW,Crystal Palace,27,1992,30,29,2612,...,85.0,19.0,24.0,4.0,0.0,0.0,186.0,2.0,49.0,3.9
6,Raheem Sterling\b400bde0,456,eng ENG,FW,Manchester City,25,1994,31,28,2536,...,51.0,17.0,20.0,6.0,1.0,0.0,186.0,16.0,35.0,31.4


False Positives

In [233]:
df_FP

,Player,y2020-21_Rk,y2020-21_Nation,y2020-21_Pos,y2020-21_Squad,y2020-21_Age,y2020-21_Born,y2020-21_MP,y2020-21_Starts,y2020-21_Min,...,y2017-18_Crs.1,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%
0,Paul Pogba\867239d3,381,fr FRA,MFFW,Manchester Utd,27,1993,26,21,1897,...,34.0,33.0,20.0,0.0,0.0,0.0,246.0,58.0,39.0,59.8


True Negatives

In [234]:
df_TN

,Player,y2020-21_Rk,y2020-21_Nation,y2020-21_Pos,y2020-21_Squad,y2020-21_Age,y2020-21_Born,y2020-21_MP,y2020-21_Starts,y2020-21_Min,...,y2017-18_Crs.1,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%
0,Olivier Giroud\16ceb862,178,fr FRA,FW,Chelsea,33,1986,17,8,748,...,4.0,1.0,4.0,0.0,0.0,0.0,12.0,24.0,25.0,49.0
1,Oliver McBurnie\78b8ac63,309,sct SCO,FW,Sheffield Utd,24,1996,23,12,1324,...,2.0,0.0,2.0,0.0,0.0,0.0,15.0,29.0,26.0,52.7
2,Islam Slimani\ffec9769,445,dz ALG,FW,Leicester City,32,1988,1,0,20,...,3.0,0.0,2.0,0.0,0.0,0.0,16.0,16.0,12.0,57.1
3,Alex Oxlade-Chamberlain\38c7feef,364,eng ENG,FWMF,Liverpool,26,1993,13,2,253,...,13.0,10.0,2.0,0.0,0.0,0.0,26.0,3.0,5.0,37.5
4,Theo Walcott\1ecb65be,499,eng ENG,FW,Everton,31,1989,1,0,13,...,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0
5,Ruben Loftus-Cheek\e97fd090,275,eng ENG,MFFW,Fulham,24,1996,30,21,1909,...,12.0,26.0,23.0,1.0,0.0,0.0,183.0,32.0,64.0,33.3
6,Cenk Tosun\9a104dee,478,tr TUR,FW,Everton,29,1991,5,0,48,...,6.0,5.0,2.0,0.0,0.0,0.0,56.0,42.0,85.0,33.1
7,Matt Phillips\7bb5f40e,375,sct SCO,MFFW,West Brom,29,1991,33,20,1828,...,97.0,24.0,19.0,0.0,0.0,0.0,158.0,37.0,49.0,43.0
8,Alex Oxlade-Chamberlain\38c7feef,364,eng ENG,FWMF,Liverpool,26,1993,13,2,253,...,40.0,20.0,23.0,0.0,0.0,0.0,153.0,16.0,31.0,34.0
9,Adam Lallana\99813635,258,eng ENG,MFFW,Brighton,32,1988,30,16,1596,...,6.0,5.0,3.0,0.0,0.0,0.0,32.0,3.0,6.0,33.3


False Negatives

In [235]:
df_FN

,Player,y2020-21_Rk,y2020-21_Nation,y2020-21_Pos,y2020-21_Squad,y2020-21_Age,y2020-21_Born,y2020-21_MP,y2020-21_Starts,y2020-21_Min,...,y2017-18_Crs.1,y2017-18_Int.1,y2017-18_TklW.1,y2017-18_Pkwon,y2017-18_PKcon,y2017-18_OG,y2017-18_Recov,y2017-18_Won,y2017-18_Lost,y2017-18_Won%
0,Callum Wilson\c596fcb0,519,eng ENG,FW,Newcastle Utd,28,1992,26,23,2084,...,13.0,5.0,10.0,0.0,0.0,0.0,105.0,35.0,115.0,23.3
1,Danny Welbeck\ce5143da,508,eng ENG,FW,Brighton,29,1990,24,17,1545,...,4.0,12.0,10.0,0.0,0.0,0.0,55.0,25.0,39.0,39.1
2,Pierre-Emerick Aubameyang\d5dd5f1f,32,ga GAB,FW,Arsenal,31,1989,29,26,2332,...,21.0,6.0,7.0,0.0,0.0,0.0,47.0,8.0,28.0,22.2
3,Harvey Barnes\3ea50f67,47,eng ENG,FWMF,Leicester City,22,1997,25,22,1945,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,100.0
4,Jesse Lingard\810e3c74,270,eng ENG,MFFW,West Ham,27,1992,16,16,1421,...,25.0,14.0,19.0,0.0,0.0,0.0,128.0,9.0,42.0,17.6


Predicting 23/24 using 17/18

In [242]:
df_20 = df_full.iloc[:, : 151]
df_20

,Player,y2020-21_Rk,y2020-21_Nation,y2020-21_Pos,y2020-21_Squad,y2020-21_Age,y2020-21_Born,y2020-21_MP,y2020-21_Starts,y2020-21_Min,...,y2020-21_Crs.1,y2020-21_Int.1,y2020-21_TklW.1,y2020-21_Pkwon,y2020-21_PKcon,y2020-21_OG,y2020-21_Recov,y2020-21_Won,y2020-21_Lost,y2020-21_Won%
0,Aaron Connolly\27c01749,104,ie IRL,FW,Brighton,20,2000,17,9,791,...,2,0,5,1,0,0,28,11,30,26.8
1,Aaron Cresswell\4f974391,110,eng ENG,DF,West Ham,30,1989,36,36,3170,...,91,36,13,0,0,0,307,38,30,55.9
2,Aaron Ramsdale\466fb2c5,386,eng ENG,GK,Sheffield Utd,22,1998,38,38,3420,...,0,0,0,0,1,0,183,0,0,NaN
3,Aaron Wan-Bissaka\9e525177,502,eng ENG,DF,Manchester Utd,22,1997,34,34,3060,...,57,66,52,0,0,0,267,46,35,56.8
4,Abdoulaye DoucourÃÂ©\02b29014,137,fr FRA,MF,Everton,27,1993,29,29,2443,...,10,35,42,0,0,0,304,37,26,58.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,Zack Steffen\42130443,453,us USA,GK,Manchester City,25,1995,1,1,90,...,0,0,0,0,0,0,2,0,0,NaN
539,ÃÂaÃÂlar SÃÂ¶yÃÂ¼ncÃÂ¼\21166ff4,452,tr TUR,DF,Leicester City,24,1996,23,19,1819,...,8,27,16,0,0,0,186,62,37,62.6
540,ÃÂrik Lamela\abe66106,259,ar ARG,FWMF,Tottenham,28,1992,23,5,717,...,9,4,11,0,0,0,48,5,17,22.7
541,ÃÂ°lkay GÃÂ¼ndoÃÂan\819b3158,190,de GER,MF,Manchester City,29,1990,28,23,2029,...,19,19,18,1,0,0,170,14,16,46.7


In [243]:
players = df_20['Player']
players

0                     Aaron Connolly\27c01749
1                    Aaron Cresswell\4f974391
2                     Aaron Ramsdale\466fb2c5
3                  Aaron Wan-Bissaka\9e525177
4              Abdoulaye DoucourÃÂ©\02b29014
                        ...                  
538                     Zack Steffen\42130443
539    ÃÂaÃÂlar SÃÂ¶yÃÂ¼ncÃÂ¼\21166ff4
540                   ÃÂrik Lamela\abe66106
541          ÃÂ°lkay GÃÂ¼ndoÃÂan\819b3158
542           Ã
Âukasz FabiaÃ
Âski\9328b835
Name: Player, Length: 543, dtype: object

In [244]:

df_20 = df_20.drop(['y2020-21_Nation',
                                'y2020-21_Pos',
                                'y2020-21_Squad',
                                'Player',
                                'y2020-21_Rk']
                                ,1)
df_20

,y2020-21_Age,y2020-21_Born,y2020-21_MP,y2020-21_Starts,y2020-21_Min,y2020-21_90s,y2020-21_Gls,y2020-21_Ast,y2020-21_G-PK,y2020-21_PK,...,y2020-21_Crs.1,y2020-21_Int.1,y2020-21_TklW.1,y2020-21_Pkwon,y2020-21_PKcon,y2020-21_OG,y2020-21_Recov,y2020-21_Won,y2020-21_Lost,y2020-21_Won%
0,20,2000,17,9,791,8.8,2,1,2,0,...,2,0,5,1,0,0,28,11,30,26.8
1,30,1989,36,36,3170,35.2,0,8,0,0,...,91,36,13,0,0,0,307,38,30,55.9
2,22,1998,38,38,3420,38.0,0,0,0,0,...,0,0,0,0,1,0,183,0,0,NaN
3,22,1997,34,34,3060,34.0,2,4,2,0,...,57,66,52,0,0,0,267,46,35,56.8
4,27,1993,29,29,2443,27.1,2,3,2,0,...,10,35,42,0,0,0,304,37,26,58.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,25,1995,1,1,90,1.0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,NaN
539,24,1996,23,19,1819,20.2,1,0,1,0,...,8,27,16,0,0,0,186,62,37,62.6
540,28,1992,23,5,717,8.0,1,0,1,0,...,9,4,11,0,0,0,48,5,17,22.7
541,29,1990,28,23,2029,22.5,13,2,12,1,...,19,19,18,1,0,0,170,14,16,46.7


In [245]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_20_normalized = pd.DataFrame(scaler.fit_transform(df_20), columns= df_20.columns)

#df_20_normalized.dropna()
df_20_normalized

,y2020-21_Age,y2020-21_Born,y2020-21_MP,y2020-21_Starts,y2020-21_Min,y2020-21_90s,y2020-21_Gls,y2020-21_Ast,y2020-21_G-PK,y2020-21_PK,...,y2020-21_Crs.1,y2020-21_Int.1,y2020-21_TklW.1,y2020-21_Pkwon,y2020-21_PKcon,y2020-21_OG,y2020-21_Recov,y2020-21_Won,y2020-21_Lost,y2020-21_Won%
0,0.181818,0.826087,0.432432,0.236842,0.231062,0.231579,0.086957,0.071429,0.105263,0.000000,...,0.010989,0.000000,0.060976,0.2,0.000000,0.0,0.057971,0.048246,0.139535,0.268
1,0.636364,0.347826,0.945946,0.947368,0.926879,0.926316,0.000000,0.571429,0.000000,0.000000,...,0.500000,0.461538,0.158537,0.0,0.000000,0.0,0.635611,0.166667,0.139535,0.559
2,0.272727,0.739130,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.378882,0.000000,0.000000,NaN
3,0.272727,0.695652,0.891892,0.894737,0.894706,0.894737,0.086957,0.285714,0.105263,0.000000,...,0.313187,0.846154,0.634146,0.0,0.000000,0.0,0.552795,0.201754,0.162791,0.568
4,0.500000,0.521739,0.756757,0.763158,0.714244,0.713158,0.086957,0.214286,0.105263,0.000000,...,0.054945,0.448718,0.512195,0.0,0.000000,0.0,0.629400,0.162281,0.120930,0.587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,0.409091,0.608696,0.000000,0.026316,0.026031,0.026316,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.004141,0.000000,0.000000,NaN
539,0.363636,0.652174,0.594595,0.500000,0.531734,0.531579,0.043478,0.000000,0.052632,0.000000,...,0.043956,0.346154,0.195122,0.0,0.000000,0.0,0.385093,0.271930,0.172093,0.626
540,0.545455,0.478261,0.594595,0.131579,0.209418,0.210526,0.043478,0.000000,0.052632,0.000000,...,0.049451,0.051282,0.134146,0.0,0.000000,0.0,0.099379,0.021930,0.079070,0.227
541,0.590909,0.391304,0.729730,0.605263,0.593156,0.592105,0.565217,0.142857,0.631579,0.111111,...,0.104396,0.243590,0.219512,0.2,0.000000,0.0,0.351967,0.061404,0.074419,0.467


In [247]:
y_pred = neigh.predict(df_20_normalized)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,

In [262]:
df_20_normalized = pd.DataFrame(players)
df_20_normalized['Label'] = y_pred
df_20_normalized

,Player,Label
0,Aaron Connolly\27c01749,0
1,Aaron Cresswell\4f974391,0
2,Aaron Ramsdale\466fb2c5,0
3,Aaron Wan-Bissaka\9e525177,0
4,Abdoulaye DoucourÃÂ©\02b29014,0
...,...,...
538,Zack Steffen\42130443,0
539,ÃÂaÃÂlar SÃÂ¶yÃÂ¼ncÃÂ¼\21166ff4,0
540,ÃÂrik Lamela\abe66106,0
541,ÃÂ°lkay GÃÂ¼ndoÃÂan\819b3158,1


In [264]:
df_20_normalized[df_20_normalized['Label']==1]

,Player,Label
72,Bruno Fernandes\507c7bdf,1
77,Callum Wilson\c596fcb0,1
87,Che Adams\f2bf1b0f,1
90,Chris Wood\4e9a0555,1
134,Dominic Calvert-Lewin\59e6e5bf,1
174,Gabriel Jesus\b66315ae,1
189,Harry Kane\21a66f6a,1
208,Jack Harrison\aa849a12,1
230,Jamie Vardy\45963054,1
294,Kevin De Bruyne\e46012d4,1


In [265]:
df_20_normalized[df_20_normalized['Label']==0]

,Player,Label
0,Aaron Connolly\27c01749,0
1,Aaron Cresswell\4f974391,0
2,Aaron Ramsdale\466fb2c5,0
3,Aaron Wan-Bissaka\9e525177,0
4,Abdoulaye DoucourÃÂ©\02b29014,0
...,...,...
537,Yves Bissouma\6c203af0,0
538,Zack Steffen\42130443,0
539,ÃÂaÃÂlar SÃÂ¶yÃÂ¼ncÃÂ¼\21166ff4,0
540,ÃÂrik Lamela\abe66106,0
